# The Algorithm - Regression Trees
- Assuming all feature values are quantitative.  
- The below algorithm is to build a decision stump i.e. a decision tree with only one split.

### Init Stump Variables

- Split Variable $\rightarrow$ Where the variable on which the split is made will be stored
- RSS Value $\rightarrow$ The RSS value of the split
- Split Threshold $\rightarrow$ The threshold against which the split is made
- MSE $\rightarrow$ Meean Squared Error of the split
- Predictions $\rightarrow$ Prediction value for each region

$$RSS = \sum_{j = 1}^J \sum_{i \in R_j} (y_i - y_{R_j})^2$$  
$$MSE = \frac{1}{n} RSS$$

- Iterate over all feature variables
    - Calculate the `min` and `max` value of each feature
    - Generate `n` linearly spaced threshold values between `[min, max]` that will act as `thresholds`
    - Iterate over the `thresholds`
        - Split the attributes in `2-regions = [R1, R2]`, where `R1 = [feature < threshold]` and `R2 = [feature >= threshold]`
        - Calculate `RSS` for each region
        - Update the initialized variables
        
    - At the end of the iteration the initialized variables will hold:
        - An attribute of `X`
        - A `threshold` value, that divides the feature space into 2-regions based on the minimum RSS value which indicates the best split.
        - Each region will have it's corresponding prediction
    
**Note:** For a decision tree, the above steps are recursively performed for each subsequent regions, until a stopping criterion is met. This whole structure is easily represented in a tree data-structure; hence the name `decision tree`.

In [1]:
import numpy as np
import pandas as pd
from pprint import pprint
import random
random.seed(518123)

In [2]:
data = pd.read_csv('Boston.csv')

In [3]:
data.head()

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,b,lstat,medv
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2


## Building a Decision Stump

In [4]:
def decision_stump(features, labels, num_thresholds=200):
    
    # Init Stump Variables
    stump = {'split_variable': "",
             'split_threshold': None,
             'rss_split': None,
             'mse_split': np.inf,
             'split_predictions': {'lower': None, 'higher': None}}
    
    # Iterate over all feature variables
    for column in features.columns:
        # Calculate the `min` and `max` value of each feature
        min_threshold, max_threshold = features[column].min(), features[column].max()
        
        # Generate `n` linearly spaced threshold values between `[min, max]` that will act as `thresholds`
        thresholds = np.arange(min_threshold, max_threshold, step=(max_threshold-min_threshold)/(num_thresholds-1))
        
        # Iterate over the `thresholds`
        for threshold in thresholds:
            # Split the attributes in `2-regions = [R1, R2]`, where `R1 = [feature < threshold]` and `R2 = [feature >= threshold]`
            less_than_idx = (features[features[column] < threshold]).index
            greater_than_idx = (features[features[column] >= threshold]).index
            
            # Calculate Predictions for each region
            pred_less_than = labels.loc[less_than_idx].mean()
            pred_greater_than = labels.loc[greater_than_idx].mean()
            
            # Calculate RSS
            RSS = ((labels.loc[less_than_idx] - pred_less_than)**2).sum() + ((labels.loc[greater_than_idx] - pred_greater_than)**2).sum()
            
            # Update stump attributes
            mse = (RSS / len(labels)).values[0]
            
            if stump['rss_split'] is not None:
                if mse < stump['mse_split']:
                    stump.update({'split_variable': column,
                                  'split_threshold': threshold,
                                  'rss_split': RSS.values[0],
                                  'mse_split': mse,
                                  'split_predictions': {'lower': pred_less_than.values[0],
                                                        'higher': pred_greater_than.values[0]}})
            else:
                stump.update({'split_variable': column,
                                  'split_threshold': threshold,
                                  'rss_split': RSS.values[0],
                                  'mse_split': mse,
                                  'split_predictions': {'lower': pred_less_than.values[0],
                                                        'higher': pred_greater_than.values[0]}})
    return stump

In [5]:
X = data['rm lstat'.split()]
Y = data[['medv']]
stump = decision_stump(X, Y)
pprint(stump)

{'mse_split': 46.33181458857348,
 'rss_split': 23443.898181818182,
 'split_predictions': {'higher': 37.099999999999994,
                       'lower': 19.91818181818183},
 'split_threshold': 6.917944723618085,
 'split_variable': 'rm'}


## Predicting using Decision Stump